# Titanic Dataset – Data Cleaning & Feature Engineering

Este proyecto realiza un proceso completo de **limpieza, transformación y análisis de datos**
sobre el dataset del Titanic utilizando **Python y pandas**.

El objetivo es preparar los datos para un análisis posterior, aplicando técnicas habituales
de data cleaning, feature engineering y lógica de negocio.


## Objetivos del proyecto

- Analizar y detectar valores nulos
- Aplicar diferentes estrategias de imputación
- Limpiar columnas de texto mediante expresiones regulares
- Crear nuevas variables (feature engineering)
- Realizar análisis numérico y rankings
- Calcular métricas personalizadas por pasajero


In [4]:
import pandas as pd
from unidecode import unidecode


## Carga del dataset

Cargamos el dataset original del Titanic desde la carpeta `data/`. 
Esto nos permitirá trabajar con los datos en pandas y realizar la limpieza 
y análisis posteriores. Mostramos las primeras filas para comprobar la estructura.


In [9]:
df = pd.read_excel('../data/Titanic.xlsx')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Carga del dataset

Hemos cargado el dataset **Titanic.xlsx** desde la carpeta `data/` utilizando **pandas**.  

La función `pd.read_excel()` permite leer archivos de Excel directamente en un DataFrame de pandas.  
El método `head()` nos muestra las primeras cinco filas para verificar que la carga se haya realizado correctamente y que los nombres de las columnas y los tipos de datos son los esperados.


## Comprobación de valores nulos

El primer paso en la limpieza de datos es identificar los valores faltantes (nulos) en el dataset.  

- Creamos un **DataFrame booleano** que indique si cada celda tiene un valor nulo (`True`) o no (`False`).  
- Contamos los **valores nulos por columna** y el **total de valores nulos en todo el DataFrame** para tener una visión general del estado de los datos.


In [10]:
# DataFrame booleano indicando valores nulos
valores_nulos = df.isnull()
valores_nulos.head()

# Total de valores nulos por columna
nulos_por_columna = df.isnull().sum()
print("Valores nulos por columna:\n", nulos_por_columna)

# Total de valores nulos en todo el DataFrame
total_nulos = df.isnull().sum().sum()
print("\nTotal de valores nulos en el DataFrame:", total_nulos)


Valores nulos por columna:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Total de valores nulos en el DataFrame: 866


### Resultados de la comprobación de valores nulos

Tras ejecutar la comprobación:

- La columna **Age** tiene 177 valores nulos.  
- La columna **Cabin** es la que más valores nulos presenta, con 687 registros faltantes.  
- La columna **Embarked** tiene 2 valores nulos.  
- El total de valores nulos en el DataFrame es de **866**.

Esto nos indica que debemos aplicar estrategias de imputación o limpieza para estas columnas antes de continuar con el análisis.


### Relleno de valores nulos (Imputación)

Aplicaremos diferentes estrategias según la columna:

- **Age:** rellenaremos los valores nulos con la media de la columna.  
- **Fare:** rellenaremos los valores nulos con un valor constante (100).  
- **Embarked:** rellenaremos los valores nulos con la moda.  
- **Cabin:** aplicaremos primero un relleno hacia adelante (ffill) y luego hacia atrás (bfill).


In [12]:
# Relleno de Age con la media
df['Age'].fillna(round(df['Age'].mean(), 0), inplace=True)

# Relleno de Fare con valor constante
df['Fare'].fillna(100, inplace=True)

# Relleno de Embarked con la moda
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Relleno de Cabin hacia adelante y hacia atrás
df['Cabin'].fillna(method='ffill', inplace=True)
df['Cabin'].fillna(method='bfill', inplace=True)

# Comprobación de valores nulos después del relleno
df.isnull().sum()


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11796\4127617805.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(round(df['Age'].mean(), 0), inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_11796\4127617805.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### Resultado final del relleno de valores nulos

Después de aplicar las estrategias de imputación:

- **Age:** se reemplazaron los valores nulos por la media de la columna.  
- **Fare:** se reemplazaron los valores nulos por 100.  
- **Embarked:** se reemplazaron los valores nulos por la moda.  
- **Cabin:** se rellenaron primero hacia adelante (ffill) y luego hacia atrás (bfill).  

Ahora, **ninguna columna tiene valores nulos**, como se observa en el conteo final de valores nulos:



## Limpieza de texto y normalización de columnas

En este paso:

- Eliminamos acentos y caracteres especiales de los nombres de las columnas.  
- Eliminamos espacios al inicio y al final de los nombres de las columnas.  
- Convertimos todos los nombres de columnas a minúsculas para mayor consistencia y facilidad de uso.

Esto garantiza que las columnas sean más fáciles de manipular y evita errores por diferencias de formato.


In [13]:
# Limpiar columnas de texto y normalizar nombres
df.columns = [unidecode(col) for col in df.columns]  # Eliminar acentos
df.columns = df.columns.str.strip()                  # Eliminar espacios al inicio y final
df.columns = df.columns.str.lower()                 # Convertir a minúsculas

# Mostrar las primeras filas para verificar los cambios
df.head()


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C85,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C85,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,C123,S


Las columnas han sido limpiadas y normalizadas correctamente:

- Los nombres de las columnas ya no tienen acentos ni caracteres especiales.
- Se eliminaron espacios al inicio y final de los nombres.
- Todos los nombres de columnas están en minúsculas, facilitando futuras manipulaciones y evitando errores por inconsistencias de formato.


## Filtrado avanzado de datos

Filtraremos los pasajeros cumpliendo estas condiciones:

- Edad entre 18 y 60 años.
- Fare (tarifa) mayor al percentil 50.

Además, crearemos una nueva columna `Categoria_Edad` con las siguientes reglas:

- Menor de 30 años → "Joven"
- Entre 30 y 45 años → "Adulto"
- Mayor de 45 años → "Mayor"


In [14]:
# Filtrar pasajeros por rango de edad y fare
df_filtered = df.loc[(df['age'].between(18, 60)) & (df['fare'] > df['fare'].quantile(0.50))].copy()

# Crear columna Categoria_Edad
df_filtered.loc[df_filtered['age'] < 30, 'Categoria_Edad'] = 'Joven'
df_filtered.loc[df_filtered['age'].between(30, 45), 'Categoria_Edad'] = 'Adulto'
df_filtered.loc[df_filtered['age'] > 45, 'Categoria_Edad'] = 'Mayor'

# Mostrar las primeras filas del DataFrame filtrado
df_filtered.head()


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,Categoria_Edad
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adulto
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adulto
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mayor
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,Mayor
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,C103,S,Adulto


Podemos observar que el DataFrame ha sido filtrado correctamente:

- Solo se incluyen pasajeros con edad entre 18 y 60 años y con tarifa (`Fare`) superior al percentil 50.
- La nueva columna `Categoria_Edad` clasifica a los pasajeros según su edad:
    - "Joven" si edad < 30
    - "Adulto" si edad entre 30 y 45
    - "Mayor" si edad > 45


## Análisis numérico: ranking de tarifas

En esta celda vamos a:

1. **Ordenar** el DataFrame por la columna `Fare` en orden descendente, para identificar a los pasajeros que pagaron las tarifas más altas.
2. **Eliminar duplicados** basados en `PassengerId` y `Pclass`, manteniendo solo la primera aparición, para evitar conteos duplicados.
3. **Crear una nueva columna `Fare_Rank`** que asigne un ranking descendente a cada pasajero según su tarifa.

Esto nos permitirá analizar más fácilmente las tarifas y preparar los datos para análisis posteriores y cálculo de puntuaciones.


In [15]:
# Ordenamos por tarifa descendente y eliminamos duplicados por PassengerId y Pclass
df = df.sort_values('fare', ascending=False).drop_duplicates(['passengerid', 'pclass'], keep='first')

# Creamos la columna Fare_Rank con el ranking descendente de Fare
df['Fare_Rank'] = df['fare'].rank(ascending=False)

# Mostramos las primeras filas para comprobar
df.head()


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,Fare_Rank
679,680,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C,2.0
258,259,1,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,B77,C,2.0
737,738,1,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C,2.0
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0000,C23 C25 C27,S,5.5
438,439,0,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0000,C23 C25 C27,S,5.5


### Análisis del ranking de tarifas

La columna **Fare_Rank** indica la posición de cada pasajero en función de su tarifa (**Fare**) en orden descendente.  
Los pasajeros con tarifas más altas obtienen un ranking menor (1 es el más alto).  
Esto nos permite identificar rápidamente quién pagó más y comparar pasajeros en términos de gasto.
